In [1]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

from dotenv import load_dotenv

import pandas as pd
import os

In [3]:
load_dotenv()
MISTRAL_API = os.getenv("MISTRAL_API")

client = MistralClient(api_key=MISTRAL_API)

In [4]:
df = pd.read_csv("../data/reseted_test_predictions.csv")
df.head()

,index_x,artist,song,tag,Lyric,tag_2,True_end_lyrics,index_y,Generated Lyrics
0,0,5 Seconds of Summer,No Shame,[Verse 1: Luke],"Angel, with the gun in your hand",[Verse],"Pointin' my direction, givin' me affection Lov...",0,"Angel, with the gun in your hand, looking back..."
1,1,5 Seconds of Summer,No Shame,"[Pre-Chorus: Luke, Luke & Ashton]",Go on and light me like a cigarette,[Pre-Chorus],Even if it might be somethin' you regret You g...,1,Go on and light me like a cigarette as I am ma...
2,2,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin' Neve...,[Chorus],a reaction Changin' my face and callin' it fas...,2,I only light up when cameras are flashin' Neve...
3,3,5 Seconds of Summer,No Shame,[Verse 2: Luke],Lay me in the palm of your hand I'll give you ...,[Verse],somethin' sweeter than the words I left in you...,3,Lay me in the palm of your hand I'll give you ...
4,4,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin' Neve...,[Chorus],a reaction Changin' my face and callin' it fas...,4,I only light up when cameras are flashin' Neve...


In [5]:
df["Generated Lyrics"] = df["Generated Lyrics"].map(
    lambda x: x.replace("<|endoftext|>", "...")
)

In [6]:
df["Original Lyrics"] = df["Lyric"] + " " + df["True_end_lyrics"]
df.head()

,index_x,artist,song,tag,Lyric,tag_2,True_end_lyrics,index_y,Generated Lyrics,Original Lyrics
0,0,5 Seconds of Summer,No Shame,[Verse 1: Luke],"Angel, with the gun in your hand",[Verse],"Pointin' my direction, givin' me affection Lov...",0,"Angel, with the gun in your hand, looking back...","Angel, with the gun in your hand Pointin' my d..."
1,1,5 Seconds of Summer,No Shame,"[Pre-Chorus: Luke, Luke & Ashton]",Go on and light me like a cigarette,[Pre-Chorus],Even if it might be somethin' you regret You g...,1,Go on and light me like a cigarette as I am ma...,Go on and light me like a cigarette Even if it...
2,2,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin' Neve...,[Chorus],a reaction Changin' my face and callin' it fas...,2,I only light up when cameras are flashin' Neve...,I only light up when cameras are flashin' Neve...
3,3,5 Seconds of Summer,No Shame,[Verse 2: Luke],Lay me in the palm of your hand I'll give you ...,[Verse],somethin' sweeter than the words I left in you...,3,Lay me in the palm of your hand I'll give you ...,Lay me in the palm of your hand I'll give you ...
4,4,5 Seconds of Summer,No Shame,[Chorus: All],I only light up when cameras are flashin' Neve...,[Chorus],a reaction Changin' my face and callin' it fas...,4,I only light up when cameras are flashin' Neve...,I only light up when cameras are flashin' Neve...


In [7]:
data = df.sample(250, random_state=0)
data.head()

,index_x,artist,song,tag,Lyric,tag_2,True_end_lyrics,index_y,Generated Lyrics,Original Lyrics
453,559,Kesha,Take It Off,[Chorus],There's a place downtown where the freaks all ...,[Chorus],me on when they take it off When they take it ...,453,There's a place downtown where the freaks all ...,There's a place downtown where the freaks all ...
85,100,Bebe Rexha,The Way I Are (Dance with Somebody),[Chorus: Bebe Rexha],I just wanna dance with somebody I just wanna ...,[Chorus],are Just love me the way I are I just wanna da...,85,I just wanna dance with somebody I just wanna ...,I just wanna dance with somebody I just wanna ...
545,680,Meghan Trainor,Title,[Chorus],"Baby, don't call me your friend If I hear that...",[Chorus],hard to find But if you think you wanna try Th...,545,"Baby, don't call me your friend If I hear that...","Baby, don't call me your friend If I hear that..."
312,382,Halsey,Bad at Love,[Bridge],I know that you're afraid I'm gonna walk away ...,[Bridge],fades Each time the feelin' fades I know that ...,312,I know that you're afraid I'm gonna walk away ...,I know that you're afraid I'm gonna walk away ...
334,410,James Arthur,Naked,[Chorus],I'm not gonna wait until you're gone 'Cause yo...,[Chorus],pride) I'm standing here naked (I'm standing h...,334,I'm not gonna wait until you're gone 'Cause yo...,I'm not gonna wait until you're gone 'Cause yo...


In [8]:
def generate_promt(tag, lyrics):
    return f"""ONLY THE SCORE.
    Give me the score from 0/10 coherence, musicality and quality. 
    Example: 
    coherence X/10
    musicality y/10
    quality z/10
    of this {tag} of a pop song: 
    {lyrics}"""

In [9]:
original_promts = [
    generate_promt(row["tag_2"], row["Original Lyrics"]) for _, row in data.iterrows()
]
generated_promts = [
    generate_promt(row["tag_2"], row["Generated Lyrics"]) for _, row in data.iterrows()
]
generated_promts

["ONLY THE SCORE.\n    Give me the score from 0/10 coherence, musicality and quality. \n    Example: \n    coherence X/10\n    musicality y/10\n    quality z/10\n    of this [Chorus] of a pop song: \n    There's a place downtown where the freaks all come around It's a hole in the wall, it's a dirty free-for-all And they turn me on when they take it off When they take it off, everybody take it off There's a place I know if you're lookin' for a show Where they go hardcore and there's glitter on the floor And they turn me on when they take it off, everyone take it off\r\n'Cause that's where it's at? What if I just lookin' for...",
 "ONLY THE SCORE.\n    Give me the score from 0/10 coherence, musicality and quality. \n    Example: \n    coherence X/10\n    musicality y/10\n    quality z/10\n    of this [Chorus] of a pop song: \n    I just wanna dance with somebody I just wanna dance with somebody It could be anybody Tell me, are you that somebody? Don't matter who you are You can just tell

In [10]:
def send_message_mistral(
    client: MistralClient, message: str, mistral_model="open-mistral-7b"
) -> str:
    """
    Send a message to the Mistral API and return the response.

    Parameters:
        client: Mistral API client.
        message: The message to send to the API.

    Returns:
        str: Model response.
    """
    messages = [ChatMessage(role="user", content=message)]
    chat_response = client.chat(
        model=mistral_model,
        messages=messages,
    )
    return chat_response.choices[0].message.content

In [11]:
send_message_mistral(client, "What is the best French cheese?")

'It\'s subjective to determine the "best" French cheese as it largely depends on personal preferences. Here are a few famous French cheeses that are widely appreciated for their distinct flavors and textures:\n\n1. Roquefort: A blue-veined cheese made from sheep\'s milk, originating from the Massif Central region in France. It\'s known for its strong, pungent aroma and tangy, savory taste.\n\n2. Camembert: A soft, creamy cheese with a white rind, originally from Normandy. It has a rich, earthy flavor with hints of mushroom and a slight tang.\n\n3. Comté: A nutty-tasting, firm cheese from the Franche-Comté region in eastern France. It has a complex, slightly sweet flavor with a smooth, buttery texture.\n\n4. Brie de Meaux: A soft, creamy cheese with a white, bloomy rind, produced in the Marne region. It has a mild, buttery flavor with hints of fresh cream and a slightly earthy aroma.\n\n5. Munster: A soft, smelly cheese with an orange rind, originating from the Alsace region. It has a s

In [12]:
original_eval = [send_message_mistral(client, prompt) for prompt in original_promts]
generated_eval = [send_message_mistral(client, prompt) for prompt in generated_promts]

In [16]:
data["Generated_eval"] = generated_eval
data["Original_eval"] = original_eval

data

,index_x,artist,song,tag,Lyric,tag_2,True_end_lyrics,index_y,Generated Lyrics,Original Lyrics,Generated_eval,Original_eval
453,559,Kesha,Take It Off,[Chorus],There's a place downtown where the freaks all ...,[Chorus],me on when they take it off When they take it ...,453,There's a place downtown where the freaks all ...,There's a place downtown where the freaks all ...,coherence: 8/10 (The lyrics follow a clear pat...,coherence: 7/10 (The lyrics follow a clear pat...
85,100,Bebe Rexha,The Way I Are (Dance with Somebody),[Chorus: Bebe Rexha],I just wanna dance with somebody I just wanna ...,[Chorus],are Just love me the way I are I just wanna da...,85,I just wanna dance with somebody I just wanna ...,I just wanna dance with somebody I just wanna ...,coherence: 6/10 (The lyrics are understandable...,coherence: 9/10 (The lyrics are clear and easy...
545,680,Meghan Trainor,Title,[Chorus],"Baby, don't call me your friend If I hear that...",[Chorus],hard to find But if you think you wanna try Th...,545,"Baby, don't call me your friend If I hear that...","Baby, don't call me your friend If I hear that...",coherence: 8/10 (The lyrics follow a clear the...,coherence: 7/10 (The lyrics follow a clear and...
312,382,Halsey,Bad at Love,[Bridge],I know that you're afraid I'm gonna walk away ...,[Bridge],fades Each time the feelin' fades I know that ...,312,I know that you're afraid I'm gonna walk away ...,I know that you're afraid I'm gonna walk away ...,coherence: 9/10 (The lyrics follow a clear and...,coherence: 10/10 (The lyrics clearly express a...
334,410,James Arthur,Naked,[Chorus],I'm not gonna wait until you're gone 'Cause yo...,[Chorus],pride) I'm standing here naked (I'm standing h...,334,I'm not gonna wait until you're gone 'Cause yo...,I'm not gonna wait until you're gone 'Cause yo...,coherence: 8/10 (The lyrics convey a clear mes...,coherence: 8/10 (The lyrics follow a clear the...
...,...,...,...,...,...,...,...,...,...,...,...,...
741,904,Troye Sivan,FOOLS,[Verse 2],"Oh, our lives dont collide, Im aware of this W...",[Verse],"like stick, and I like aerosol I dont give a f...",741,"Oh, our lives dont collide, Im aware of this W...","Oh, our lives dont collide, Im aware of this W...",coherence: 8/10 - The verse follows a logical ...,coherence: 8/10 (The lyrics follow a clear and...
188,226,Conan Gray,Maniac,[Pre-Chorus],People like you always want back what they can...,[Pre-Chorus],I'm past that and you know that So you should ...,188,People like you always want back what they can...,People like you always want back what they can...,coherence: 8/10 (The lyrics follow a clear and...,coherence: 8/10 (The lyrics follow a clear and...
350,432,Jason Derulo,Whatcha Say,[Chorus: Imogen Heap & Jason Derulo],"Mm, whatcha say? Oh, that you only meant well?...",[Chorus],"only meant well? Well, of course, you did Mm, ...",350,"Mm, whatcha say? Oh, that you only meant well?...","Mm, whatcha say? Oh, that you only meant well?...","coherence: 8/10 (The repetition of ""whatcha sa...",Coherence: 7/10 (The lyrics follow a clear pat...
596,745,One Direction,Story of My Life,[Verse 1: Harry & Liam],Written in these walls are the stories that I ...,[Verse],us in her bones It seems to me that when I die...,596,Written in these walls are the stories that I ...,Written in these walls are the stories that I ...,coherence: 9/10 (The lyrics follow a clear nar...,coherence: 8/10 (The verse follows a logical p...


In [29]:
import re

pattern = r"(\w+)\:\s+(\d+)\/(\d+)"
data["Generated_punct"] = data["Generated_eval"].map(
    lambda row: {x[0]: (float(x[1]), float(x[2])) for x in re.findall(pattern, row)}
)
data["Original_punct"] = data["Original_eval"].map(
    lambda row: {x[0]: (float(x[1]), float(x[2])) for x in re.findall(pattern, row)}
)

data.head()

,index_x,artist,song,tag,Lyric,tag_2,True_end_lyrics,index_y,Generated Lyrics,Original Lyrics,Generated_eval,Original_eval,Generated_punct,Original_punct
453,559,Kesha,Take It Off,[Chorus],There's a place downtown where the freaks all ...,[Chorus],me on when they take it off When they take it ...,453,There's a place downtown where the freaks all ...,There's a place downtown where the freaks all ...,coherence: 8/10 (The lyrics follow a clear pat...,coherence: 7/10 (The lyrics follow a clear pat...,"{'coherence': (8.0, 10.0), 'musicality': (7.0,...","{'coherence': (7.0, 10.0), 'musicality': (8.0,..."
85,100,Bebe Rexha,The Way I Are (Dance with Somebody),[Chorus: Bebe Rexha],I just wanna dance with somebody I just wanna ...,[Chorus],are Just love me the way I are I just wanna da...,85,I just wanna dance with somebody I just wanna ...,I just wanna dance with somebody I just wanna ...,coherence: 6/10 (The lyrics are understandable...,coherence: 9/10 (The lyrics are clear and easy...,"{'coherence': (6.0, 10.0), 'musicality': (8.0,...","{'coherence': (9.0, 10.0), 'musicality': (8.0,..."
545,680,Meghan Trainor,Title,[Chorus],"Baby, don't call me your friend If I hear that...",[Chorus],hard to find But if you think you wanna try Th...,545,"Baby, don't call me your friend If I hear that...","Baby, don't call me your friend If I hear that...",coherence: 8/10 (The lyrics follow a clear the...,coherence: 7/10 (The lyrics follow a clear and...,"{'coherence': (8.0, 10.0), 'musicality': (7.0,...","{'coherence': (7.0, 10.0), 'musicality': (8.0,..."
312,382,Halsey,Bad at Love,[Bridge],I know that you're afraid I'm gonna walk away ...,[Bridge],fades Each time the feelin' fades I know that ...,312,I know that you're afraid I'm gonna walk away ...,I know that you're afraid I'm gonna walk away ...,coherence: 9/10 (The lyrics follow a clear and...,coherence: 10/10 (The lyrics clearly express a...,"{'coherence': (9.0, 10.0), 'musicality': (8.0,...","{'coherence': (10.0, 10.0), 'musicality': (8.0..."
334,410,James Arthur,Naked,[Chorus],I'm not gonna wait until you're gone 'Cause yo...,[Chorus],pride) I'm standing here naked (I'm standing h...,334,I'm not gonna wait until you're gone 'Cause yo...,I'm not gonna wait until you're gone 'Cause yo...,coherence: 8/10 (The lyrics convey a clear mes...,coherence: 8/10 (The lyrics follow a clear the...,"{'coherence': (8.0, 10.0), 'musicality': (9.0,...","{'coherence': (8.0, 10.0), 'musicality': (7.0,..."


In [36]:
for data_set in ("Generated", "Original"):
    for att in ["coherence", "musicality", "quality"]:
        data[f"{data_set}_{att}"] = data[f"{data_set}_punct"].map(
            lambda x: 10 * x.get(att, (1, -1))[0] / x.get(att, (1, -1))[1]
        )

In [41]:
for data_set in ("Generated", "Original"):
    data[f"{data_set}_mean"] = data[
        [f"{data_set}_{att}" for att in ["coherence", "musicality", "quality"]]
    ].mean(axis=1)
data.head()

,index_x,artist,song,tag,Lyric,tag_2,True_end_lyrics,index_y,Generated Lyrics,Original Lyrics,...,Generated_punct,Original_punct,Generated_coherence,Generated_musicality,Generated_quality,Original_coherence,Original_musicality,Original_quality,Generated_mean,Original_mean
453,559,Kesha,Take It Off,[Chorus],There's a place downtown where the freaks all ...,[Chorus],me on when they take it off When they take it ...,453,There's a place downtown where the freaks all ...,There's a place downtown where the freaks all ...,...,"{'coherence': (8.0, 10.0), 'musicality': (7.0,...","{'coherence': (7.0, 10.0), 'musicality': (8.0,...",8.0,7.0,6.0,7.0,8.0,6.0,7.0,7.000000
85,100,Bebe Rexha,The Way I Are (Dance with Somebody),[Chorus: Bebe Rexha],I just wanna dance with somebody I just wanna ...,[Chorus],are Just love me the way I are I just wanna da...,85,I just wanna dance with somebody I just wanna ...,I just wanna dance with somebody I just wanna ...,...,"{'coherence': (6.0, 10.0), 'musicality': (8.0,...","{'coherence': (9.0, 10.0), 'musicality': (8.0,...",6.0,8.0,7.0,9.0,8.0,7.0,7.0,8.000000
545,680,Meghan Trainor,Title,[Chorus],"Baby, don't call me your friend If I hear that...",[Chorus],hard to find But if you think you wanna try Th...,545,"Baby, don't call me your friend If I hear that...","Baby, don't call me your friend If I hear that...",...,"{'coherence': (8.0, 10.0), 'musicality': (7.0,...","{'coherence': (7.0, 10.0), 'musicality': (8.0,...",8.0,7.0,6.0,7.0,8.0,6.0,7.0,7.000000
312,382,Halsey,Bad at Love,[Bridge],I know that you're afraid I'm gonna walk away ...,[Bridge],fades Each time the feelin' fades I know that ...,312,I know that you're afraid I'm gonna walk away ...,I know that you're afraid I'm gonna walk away ...,...,"{'coherence': (9.0, 10.0), 'musicality': (8.0,...","{'coherence': (10.0, 10.0), 'musicality': (8.0...",9.0,8.0,7.0,10.0,8.0,9.0,8.0,9.000000
334,410,James Arthur,Naked,[Chorus],I'm not gonna wait until you're gone 'Cause yo...,[Chorus],pride) I'm standing here naked (I'm standing h...,334,I'm not gonna wait until you're gone 'Cause yo...,I'm not gonna wait until you're gone 'Cause yo...,...,"{'coherence': (8.0, 10.0), 'musicality': (9.0,...","{'coherence': (8.0, 10.0), 'musicality': (7.0,...",8.0,9.0,7.0,8.0,7.0,7.0,8.0,7.333333


In [44]:
data.to_csv("../data/results_mistral.csv", index=False)